In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import pickle
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.metrics import explained_variance_score, mean_absolute_error, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# load data
data is generated from another script with feature engineering

In [2]:
df_pred_2016 = pd.read_csv('../tmp/pred_2016.csv')
df_pred_2017 = pd.read_csv('../tmp/pred_2017.csv')

In [3]:
df = pd.read_csv('../tmp/train_full.csv', parse_dates=['transactiondate'])

In [4]:
# load feature names
feats_numeric = pickle.load(open('../tmp/feats_numeric.pkl', 'r'))
feats_categorical_as_numeric = pickle.load(open('../tmp/feats_categorical_as_numeric.pkl', 'r'))
feats_categorical = pickle.load(open('../tmp/feats_categorical.pkl', 'r'))
feats = pickle.load(open('../tmp/feats.pkl', 'r'))

In [5]:
# load train/validation/prediction labels
mask_train = pickle.load(open('../tmp/mask_train.pkl', 'r'))
mask_test = pickle.load(open('../tmp/mask_validation.pkl', 'r'))
# mask_prediction = pickle.load(open('../tmp/mask_prediction.pkl', 'r'))

# Impute missing values (Non linear models)

In [6]:
# need to impute all these with a `missing` value first
df[feats_categorical_as_numeric] = df[feats_categorical_as_numeric].fillna(-1)
df_pred_2016[feats_categorical_as_numeric] = df_pred_2016[feats_categorical_as_numeric].fillna(-1)
df_pred_2017[feats_categorical_as_numeric] = df_pred_2017[feats_categorical_as_numeric].fillna(-1)

In [7]:
# use label encoder to encode all categorical variables
for column in feats_categorical:
    tmp = pd.concat([df[column], df_pred_2016[column], df_pred_2017[column]], axis = 0)
    encoder = LabelEncoder().fit(tmp.astype(str))
    df[column] = encoder.transform(df[column].astype(str)).astype(np.int32)
    df_pred_2016[column] = encoder.transform(df_pred_2016[column].astype(str)).astype(np.int32)
    df_pred_2017[column] = encoder.transform(df_pred_2017[column].astype(str)).astype(np.int32)

In [8]:
# for numeric variables, fill with a big negative number
df[feats_numeric] = df[feats_numeric].fillna(-999)

feats_numeric_not_contain_timeseries = [x for x in feats_numeric if "_logerror" not in x and "f_num_month" not in x and "f_num_quarter" not in x]

df_pred_2016[feats_numeric_not_contain_timeseries] = df_pred_2016[feats_numeric_not_contain_timeseries].fillna(-999)
df_pred_2017[feats_numeric_not_contain_timeseries] = df_pred_2017[feats_numeric_not_contain_timeseries].fillna(-999)

# imp =  Imputer(missing_values=np.nan, strategy="median", axis=0)
# # df[feats_numeric] = imp.fit_transform(df[feats_numeric])
# for feat in feats_numeric:
#     if df[feat].isnull().sum() < df.shape[0]:
#         df[[feat]] = imp.fit_transform(df[[feat]])
#     else:
#         df[feat] = 0

# generate training testing sets

In [9]:
mask_train.reset_index(drop=True, inplace = True)
mask_test.reset_index(drop=True, inplace = True)

In [10]:
feats.remove('f_num_stddev_logerror')
feats.remove('f_num_avg_logerror')

In [11]:
# X_train = df.loc[mask_train, feats].astype(float).values
# X_test = df.loc[mask_test, feats].astype(float).values

# some extra filtering to remove outliers
mask_outlier = (df.logerror > 0.4) | (df.logerror < -0.4)
# mask_outlier = df.logerror > 999

X_train = df.loc[(mask_train & ~mask_outlier), feats].astype(float).values
X_test = df.loc[mask_test, feats].astype(float).values

y_train = np.array(df.loc[(mask_train & ~mask_outlier), 'logerror'].tolist())
y_test = np.array(df.loc[mask_test, 'logerror'].tolist())


# Train models

In [12]:
np.random.seed(42)
random.seed(42)

## 1. LightGBM

In [13]:
# params = {}
# params['max_bin'] = 10
# params['learning_rate'] = 0.0021 # shrinkage_rate
# params['boosting_type'] = 'gbdt'
# params['objective'] = 'regression'
# params['metric'] = 'mae'          # or 'mae'
# params['sub_feature'] = 0.345    
# params['bagging_fraction'] = 0.85 # sub_row
# params['bagging_freq'] = 40
# params['num_leaves'] = 512        # num_leaf
# params['min_data'] = 500         # min_data_in_leaf
# params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
# params['verbose'] = 0
# params['feature_fraction_seed'] = 2
# params['bagging_seed'] = 3

General parameter tuning guide:
For Better Accuracy
Use large max_bin (may be slower)
Use small learning_rate with large num_iterations
Use large num_leaves(may cause over-fitting)
Use bigger training data
Try dart


Deal with Over-fitting
Use small max_bin
Use small num_leaves
Use min_data_in_leaf and min_sum_hessian_in_leaf
Use bagging by set bagging_fraction and bagging_freq
Use feature sub-sampling by set feature_fraction
Use bigger training data
Try lambda_l1, lambda_l2 and min_gain_to_split to regularization
Try max_depth to avoid growing deep tree

In [14]:
params = {
    # 'max_bin': [10],
    'max_depth': [100],
    'num_leaves': [32],
    'feature_fraction': [0.85],
    'bagging_fraction': [0.95],
    'bagging_freq': [8],
    'learning_rate': [0.0025],
}

In [15]:
start = time.time()
scorer = make_scorer(mean_absolute_error)
model = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                          metric='mae', 
                          verbosity=0, 
                          #verbose_eval=200, 
                          num_boost_round=200,
                          #bagging_seed=42, 
                          #feature_fraction_seed=42,
                          n_jobs = 6
                          )
grid = GridSearchCV(model, params, scoring=scorer,
                          cv=3, verbose=2, n_jobs=6)
grid.fit(X_train, y_train)
print("[INFO] randomized search took {:.2f} seconds".format(
    time.time() - start))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85 
[CV] num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85 
[CV] num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85 
[CV]  num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85, total=  10.7s
[CV]  num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85, total=  11.1s
[CV]  num_leaves=32, bagging_freq=8, learning_rate=0.0025, bagging_fraction=0.95, max_depth=100, feature_fraction=0.85, total=  11.2s


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:   12.7s finished


[INFO] randomized search took 20.87 seconds


In [17]:
# prediction for test set and prediction set
y_pred = grid.predict(X_test)
# pred_lgb = grid.predict(X_pred_2016)

In [18]:
mae = grid.score(X_test, y_test)
print("[INFO] grid search MAE: {:.6f}".format(mae))
print("[INFO] randomized search best parameters: {}".format(
    grid.best_params_))

[INFO] grid search MAE: 0.065371
[INFO] randomized search best parameters: {'num_leaves': 32, 'bagging_fraction': 0.95, 'learning_rate': 0.0025, 'bagging_freq': 8, 'max_depth': 100, 'feature_fraction': 0.85}


In [18]:
mae = grid.score(X_test, y_test)
print("[INFO] grid search MAE: {:.6f}".format(mae))
print("[INFO] randomized search best parameters: {}".format(
    grid.best_params_))

[INFO] grid search MAE: 0.064093
[INFO] randomized search best parameters: {'num_leaves': 32, 'bagging_fraction': 0.95, 'learning_rate': 0.0025, 'bagging_freq': 8, 'max_depth': 100, 'feature_fraction': 0.85}


## make prediction

In [23]:
results = pd.DataFrame()
results['ParcelId'] = df_pred_2016['ParcelId']

In [24]:
df_train = df
#for m in [201610]:
for m in [201610, 201611, 201612, 201710, 201711, 201712]:
    print "now doing prediction for {} ...".format(str(m))
    year = str(m)[:4]
    month = str(m)[4:]
    
    # avg log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('mean').to_dict()
    eval('df_pred_'+year)['f_num_monthly_avg_logerror'] = group[int(month)]
                                   
    # std dev log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('std').to_dict()
    eval('df_pred_'+year)['f_num_monthly_stddev_logerror'] = group[int(month)]

    # avg log error, quarterly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('mean').to_dict()
    eval('df_pred_'+year)['f_num_quarterly_avg_logerror'] = group[4]
    
    # std dev log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('std').to_dict()
    eval('df_pred_'+year)['f_num_quarterly_stddev_logerror'] = group[4]
    
    
    eval('df_pred_'+year)['f_num_month'] = int(month)
    eval('df_pred_'+year)['f_num_quarter'] = 4
    
    pred = grid.predict(eval('df_pred_'+year)[feats].astype(float).values)
    results[str(m)] = pred
    

now doing prediction for 201610 ...
now doing prediction for 201611 ...
now doing prediction for 201612 ...
now doing prediction for 201710 ...
now doing prediction for 201711 ...
now doing prediction for 201712 ...


In [25]:
results.shape[0] == 2985217

True

In [26]:
results.to_csv('../tmp/submission_{}.csv'.format(datetime.datetime.now().strftime('%Y%m%d_%H%M%S')), index=False, float_format = '%.5f')

## 2.Xgboost

In [238]:
# xgboost params
params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,   
    'alpha': 0.4, 
    'base_score': np.mean(y_train),
    'silent': 1
}

In [239]:
d_train = xgb.DMatrix(X_train, y_train)
d_test = xgb.DMatrix(X_test)
d_pred = xgb.DMatrix(X_pred)

In [240]:
num_boost_rounds = 150
fit = xgb.train(dict(params, silent=1), d_train, num_boost_round=num_boost_rounds)

In [241]:
# prediction for test set and prediction set
y_pred = fit.predict(d_test)
pred_xgb = fit.predict(d_pred)

In [242]:
report_test_set_performance()

MAE:0.076796


In [243]:
# when not using oct data
report_test_set_performance()

MAE:0.076796


## 3. Random Forest

In [244]:
params = {
    "max_depth": 10,
    "min_samples_split": 50,
    "n_estimators": 200
}

In [245]:
fit = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1, **params)
fit.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  2.7min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=50,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=1, warm_start=False)

In [246]:
# prediction for test set and prediction set
y_pred = fit.predict(X_test)
pred_rf = fit.predict(X_pred)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   14.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:   14.9s finished


In [247]:
report_test_set_performance()

MAE:0.076814


In [248]:
report_test_set_performance()

MAE:0.076814


## 4. Elastic Net
For linear model, need to have different missing value imputation strategy.

In [464]:
params = {
    'l1_ratio':0.7, 
}

In [465]:
fit = ElasticNetCV(cv=5, random_state=42)
fit.fit(X_train, y_train)
ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=-1,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=-1,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

In [466]:
# prediction for test set and prediction set
y_pred = fit.predict(X_test)
#pred_enet = fit.predict(X_pred)

In [24]:
def report_test_set_performance():
    #explained_variance_score(y_pred=y_pred, y_true=y_test)
    print 'MAE:{}'.format(round(mean_absolute_error(y_pred=y_pred, y_true=y_test),6))
    # pd.Series(pred_lgb_test).hist()

In [468]:
report_test_set_performance()

MAE:0.06579


## 5. CatBoost

In [20]:
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(feats_categorical):
    num_uniques = len(df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)

In [22]:
num_ensembles = 3
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=100, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    
    model.fit(
        X_train, y_train,
        #cat_features=cat_feature_inds
    )
    y_pred += model.predict(X_test)
y_pred /= num_ensembles


100%|██████████| 1/1 [00:04<00:00,  4.76s/it]


In [25]:
report_test_set_performance()

MAE:0.065385


# Ensenmble - stacking

In [469]:
model_weights = {
    'lgb': 1,
    'xgb': 0.00,
    'rf': 0.00,
    'enet': 0,
    'baseline': 0,
}

## Make predictions first for each model

In [394]:
pred_all = 0
pred_baseline = df.logerror.mean()
for model_name, model_weight in model_weights.iteritems():
    pred_all += eval('pred_'+model_name) * model_weight
pred_all = pred_all / sum(model_weights.values())

# generate submission file

In [395]:
df_submission = df[['parcelid','logerror']]
df_submission['pred'] = pred_all

In [396]:
# df_submission['201610'] = df_submission['logerror'].combine_first(df_submission['pred'])
df_submission['201610'] = df_submission['pred']
df_submission['201611'] = df_submission['pred']
df_submission['201612'] = df_submission['pred']
df_submission['201710'] = df_submission['pred']
df_submission['201711'] = df_submission['pred']
df_submission['201712'] = df_submission['pred']

In [397]:
df_submission.drop_duplicates(subset = ['parcelid'], keep='first', inplace = True)

In [398]:
del df_submission['logerror']
del df_submission['pred']

In [399]:
df_submission.shape[0] == 2985217

True

In [400]:
df_submission.to_csv('../tmp/submission_{}.csv'.format(datetime.datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)